In [6]:
import pandas as pd

import psycopg2

from dotenv import load_dotenv
import os

load_dotenv()

True

In [7]:
DATA_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx"

# with pd.ExcelFile(
#         "https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx"
# ) as xls:
#     df1 = pd.read_excel(xls)

# print(df1)

raw_data = pd.read_excel(DATA_URL, sheet_name=None)
print(raw_data)
print(raw_data.keys())

{'Year 2009-2010':        Invoice StockCode                          Description  Quantity  \
0       489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1       489434    79323P                   PINK CHERRY LIGHTS        12   
2       489434    79323W                  WHITE CHERRY LIGHTS        12   
3       489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4       489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   
...        ...       ...                                  ...       ...   
525456  538171     22271                 FELTCRAFT DOLL ROSIE         2   
525457  538171     22750         FELTCRAFT PRINCESS LOLA DOLL         1   
525458  538171     22751       FELTCRAFT PRINCESS OLIVIA DOLL         1   
525459  538171     20970   PINK FLORAL FELTCRAFT SHOULDER BAG         2   
525460  538171     21931               JUMBO STORAGE BAG SUKI         2   

               InvoiceDate  Price  Customer ID         Country  
0      2009-12-

In [107]:
df = raw_data['Year 2009-2010']

df.columns = ['invoice_id', 'product_id', 'description', 'quantity', 'datetime', 'price', 'customer_id', 'country']

df.head()

ValueError: Length mismatch: Expected axis has 9 elements, new values have 8 elements

In [108]:
df.isnull().sum()

# print(df.shape)

# # very simple N/A removal, doesn't do anything
# df = df.dropna(subset=['Invoice', 'StockCode', 'Customer ID'])

# print(df.shape)

# df.head()

invoice_id           0
product_id           0
description       2928
quantity             0
datetime             0
price                0
customer_id          0
country              0
transaction_id       0
dtype: int64

In [105]:
import uuid

df['transaction_id'] = [uuid.uuid4().int for i in range(len(df))]

df = df.sort_values('datetime', ascending=True)


In [85]:
def join_uuid(df, column_name, new_column_name = 'id'):

    unique_items = df[column_name].unique()

    invoice_id_key = pd.DataFrame({
        new_column_name: [uuid.uuid4().int for i in range(len(unique_items))],
        column_name: unique_items
    })

    return df.merge(invoice_id_key, on='datetime')


In [86]:
df['datetime'] = pd.to_datetime(df['datetime'])

df = join_uuid(df, 'datetime', 'date_id')


df.head()

,invoice_id,product_id,description,quantity,datetime,price,customer_id,country,transaction_id,date_id
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,46382311627955079117754413555228739130,80258651032386649443951405565878662550
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,33648383475244924835380630520900480719,80258651032386649443951405565878662550
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,144739360979695487532231474448363548760,80258651032386649443951405565878662550
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,310071647246353799661123517196439294193,80258651032386649443951405565878662550
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,192172135995646064168431281804048062237,80258651032386649443951405565878662550


In [104]:
# Fact Table

# transactions

df_fact = df[['transaction_id', 'invoice_id', 'product_id', 'customer_id', 'quantity', 'price']]

df_fact.head()

,transaction_id,invoice_id,product_id,customer_id,quantity,price
0,46382311627955079117754413555228739130,489434,85048,13085.0,12,6.95
1,33648383475244924835380630520900480719,489434,79323P,13085.0,12,6.75
2,144739360979695487532231474448363548760,489434,79323W,13085.0,12,6.75
3,310071647246353799661123517196439294193,489434,22041,13085.0,48,2.10
4,192172135995646064168431281804048062237,489434,21232,13085.0,24,1.25


In [102]:
# Dimension Tables

# Invoice table

invoices = df.groupby('invoice_id').agg({
    'date_id': 'first'
})

invoices['cancelled'] = invoices['invoice_id'].str.startswith('C', na=False)


,date_id,cancelled
invoice_id,,
489434,80258651032386649443951405565878662550,False
489435,44055936905604262284626874401406311141,False
489436,25892385503903812361591922270163709516,False
489437,258312839775459025863350088537555031873,False
489438,314502463497531447055827933201580356734,False
...,...,...
C538121,296831506199209878857058041743725568181,True
C538122,31500654399344823881187977178576874578,True
C538123,218739180929311276308926731693898837488,True


In [60]:

## Date Table

dates = df.groupby('date_id').agg({'datetime': 'first'})

# Create other date features

dates['year'] = dates['datetime'].dt.year
dates['quarter'] = dates['datetime'].dt.quarter
dates['month'] = dates['datetime'].dt.month
dates['day'] = dates['datetime'].dt.day_name()
dates['hour'] = dates['datetime'].dt.hour


In [75]:
## Product Table
# most recent price and description
df.groupby('product_id').agg({
        'description': 'first', 
        'price': 'first'
    }).head()



,description,price
product_id,,
10002,INFLATABLE POLITICAL GLOBE,0.85
10080,GROOVY CACTUS INFLATABLE,0.85
10109,BENDY COLOUR PENCILS,0.42
10120,DOGGY RUBBER,0.21
10125,MINI FUNKY DESIGN TAPES,1.70


In [76]:
## Customer Table
# most recent country
df.sort_values('datetime', ascending=True).groupby('customer_id').agg({
    'country': 'first'
}).head()


,country
customer_id,
12346.0,United Kingdom
12347.0,Iceland
12348.0,Finland
12349.0,Italy
12351.0,Unspecified


In [61]:
dates.head()

,datetime,year,quarter,month,day,hour
date_id,,,,,,
6158034152355357720820252969861458,2010-11-25 11:38:00,2010,4,11,Thursday,11
15722294646346758969960438694545232,2010-04-20 16:41:00,2010,2,4,Tuesday,16
26940769193698154652711004213474519,2010-12-09 16:11:00,2010,4,12,Thursday,16
33698577818766280275738224760694181,2010-04-13 14:38:00,2010,2,4,Tuesday,14
39429046460143151001341845088063967,2010-09-08 12:44:00,2010,3,9,Wednesday,12


In [26]:
df['Country'].unique()

array(['United Kingdom', 'France', 'USA', 'Belgium', 'Australia', 'EIRE',
       'Germany', 'Portugal', 'Japan', 'Denmark', 'Nigeria',
       'Netherlands', 'Poland', 'Spain', 'Channel Islands', 'Italy',
       'Cyprus', 'Greece', 'Norway', 'Austria', 'Sweden',
       'United Arab Emirates', 'Finland', 'Switzerland', 'Unspecified',
       'Malta', 'Bahrain', 'RSA', 'Bermuda', 'Hong Kong', 'Singapore',
       'Thailand', 'Israel', 'Lithuania', 'West Indies', 'Lebanon',
       'Korea', 'Brazil', 'Canada', 'Iceland'], dtype=object)

In [4]:
raw_data['Year 2009-2010'].groupby('Invoice', 'StockCode')

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom
